In [3]:
import pandas as pd
import numpy as np
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [4]:
data = pd.read_csv('uci-news-aggregator.csv')
data=data.sample(frac=1)

In [5]:
data

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
364494,364954,Mom threatened with lifetime Facebook ban over...,http://www.krmg.com/news/news/local/mom-threat...,KRMG,t,d8BWfaKuFt7FzyMDlxfPKuVhqh83M,www.krmg.com,1404758687721
420226,420745,Marketing Campaign Lessons from the ALS Ice Bu...,http://www.business2community.com/marketing/ma...,Business 2 Community,e,dasP-s3StfCucdMOvti-g0CP9NqUM,www.business2community.com,1409215448074
185870,186206,"Samsung Galaxy Tab S with fingerprint sensor, ...",http://zeenews.india.com/business/gadgets/gadg...,Zee News,t,dmgTRrgLHSMABLM-xAO4_XUn4mfLM,zeenews.india.com,1399445085475
63887,63888,Aircraft spot objects in new search area for m...,http://www.therecord.com/news-story/4438378-ai...,Waterloo Record,b,dvqkQfPMRSHcVMMZg6jQdlFk9GPWM,www.therecord.com,1396125228284
323707,324167,Side-by-Side Comparisons Show the Massive Visu...,http://www.androidheadlines.com/2014/06/side-s...,Android Headlines - Android News,t,dubwcJArLL_qAKML5LGPLiunKzNLM,www.androidheadlines.com,1403863851964
14491,14492,Chris Pine gets six month driving ban after pl...,http://www.dailymail.co.uk/tvshowbiz/article-2...,Daily Mail,e,dwXznuDZkJ7M3kM6wa_cKyGn1pnKM,www.dailymail.co.uk,1395062976897
80326,80402,Foreign banks need to be responsive to local n...,http://www.deccanherald.com/content/396033/for...,Deccan Herald,b,d80TjEemAhW18OMzRFagYuDvsaiVM,www.deccanherald.com,1396430654999
150047,150383,Reports: Charter Close to Comcast Subscriber Deal,http://www.broadcastingcable.com/news/currency...,Broadcasting \& Cable,t,dH4GX2dS0UooO5MBI3UkgYOyB0ejM,www.broadcastingcable.com,1398224850624
104295,104492,Study: Americans twice as safe eating at home,http://www.thepacker.com/fruit-vegetable-news/...,The Packer,m,dRXovvp3PWwGsiMEosO8TzJijWqBM,www.thepacker.com,1397301309960
222283,222729,Macklemore 'accidentally' dresses as hook-nose...,http://theweek.com/speedreads/index/261790/spe...,The Week Magazine,e,dHz6XtYoDONU63MF5iU46Wx3pz41M,theweek.com,1400532799420


In [6]:
# Load Google's pre-trained Word2Vec model.
w2vmodel = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [7]:
#w2vmodel.wv.most_similar(positive='compute')

In [8]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        #if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
        result.append(token) #(lemmatize_stemming(token))
    return result

In [9]:
data.shape[0]

422419

In [13]:
featX = np.zeros((data.shape[0], 300), dtype=float)
featY = np.zeros((data.shape[0]), dtype = str)
idx = 0
for index,row in data.iterrows():
    title = row[1]
    title = preprocess(title)
    vec = np.zeros(300,dtype=float)
    for w in title:
        # Get the embedding for word w
        if w not in w2vmodel: continue
        vec = w2vmodel[w]
        featX[idx] += vec
    # Average the feature vector by the length of the title
    if len(title) != 0: featX[idx] = featX[idx]/float(len(title))
        
    # Create the label for these feature vectors
    featY[idx] = row[4]
    if featY[idx] == 'b': featY[idx] = 0
    if featY[idx] == 'e': featY[idx] = 1
    if featY[idx] == 'm': featY[idx] = 2
    if featY[idx] == 't': featY[idx] = 3
    idx += 1
    #if idx == 5:break

In [14]:
len(featX)

422419

In [15]:
X_train = featX[:350000]
y_train = featY[:350000]
X_vaildate = featX[350001:]
y_validate = featY[350001:]

In [16]:
featY[50000:50030]

array(['3', '1', '1', '0', '0', '3', '1', '1', '1', '0', '1', '0', '1',
       '1', '0', '3', '0', '0', '3', '3', '1', '2', '3', '0', '3', '3',
       '3', '3', '0', '0'], dtype='<U1')

In [ ]:
from sklearn.svm import SVC 
svm_model_linear = SVC(kernel = 'linear', C = 1).fit(X_train, y_train) 
svm_predictions = svm_model_linear.predict(X_validate)

In [107]:
# model accuracy for X_test   
accuracy = svm_model_linear.score(X_test, y_test) 